## Lab 2-3. Parse an AI Model to Extract Model Information

In [ ]:
!wget https://github.com/ONNC/onnc-tutorial/raw/master/models/lenet/lenet.onnx 

The ONNX format stores an model graph as a protobuf structure and and it can be accessed using the standard protobuf protocol APIs. Protocol buffers are Google’s language-neutral, platform-neutral, extensible mechanism for serializing structured data. You define how you want your data to be structured once, then you can use special generated source code to easily write and read your structured data to and from a variety of data streams and using a variety of languages. In this section, we will use the Python API to process an ONNX model as an example.

In [ ]:
import onnx

onnx_model = onnx.load('./lenet.onnx')

# The model is represented as a protobuf structure and it can be accessed
# using the standard python-for-protobuf methods

## list all the operator types in the model
node_list = []
count = []
for i in onnx_model.graph.node:
    if (i.op_type not in node_list):
        node_list.append(i.op_type)
        count.append(1)
    else:
        idx = node_list.index(i.op_type)
        count[idx] = count[idx]+1
print(node_list)
print(count)


In [ ]:
## find the IR version
print(onnx_model.ir_version)

## find the computation graph
print(onnx_model.graph)

In [ ]:
## find the number of inputs
print(len(onnx_model.graph.input))

## find the number of nodes in the graph
print(len(onnx_model.graph.node))


In [1]:
import onnx

onnx_model = onnx.load('./lenet.onnx')
for i in onnx_model.graph.node:
    if (i.op_type == 'Conv'):
        print(i)

input: "import/Placeholder:0"
input: "import/conv1first/Variable:0"
output: "import/conv1first/Conv2D:0"
name: "import/conv1first/Conv2D"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "pads"
  ints: 2
  ints: 2
  ints: 2
  ints: 2
  type: INTS
}

input: "import/pool1/MaxPool:0"
input: "import/conv2/Variable:0"
output: "import/conv2/Conv2D:0"
name: "import/conv2/Conv2D"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "pads"
  ints: 2
  ints: 2
  ints: 2
  ints: 2
  type: INTS
}

input: "import/pool2/MaxPool:0"
input: "import/conv3/Variable:0"
output: "import/conv3/Conv2D:0"
name: "import/conv3/Conv2D"
op_type: "

### Lab 2-3-1. Extract Input Information From an ONNX Model

In [2]:
import onnx

onnx_model = onnx.load('./lenet.onnx')
for i in onnx_model.graph.node:
    if (i.op_type == 'Conv'):
        print(i)

input: "import/Placeholder:0"
input: "import/conv1first/Variable:0"
output: "import/conv1first/Conv2D:0"
name: "import/conv1first/Conv2D"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "pads"
  ints: 2
  ints: 2
  ints: 2
  ints: 2
  type: INTS
}

input: "import/pool1/MaxPool:0"
input: "import/conv2/Variable:0"
output: "import/conv2/Conv2D:0"
name: "import/conv2/Conv2D"
op_type: "Conv"
attribute {
  name: "dilations"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "strides"
  ints: 1
  ints: 1
  type: INTS
}
attribute {
  name: "kernel_shape"
  ints: 5
  ints: 5
  type: INTS
}
attribute {
  name: "pads"
  ints: 2
  ints: 2
  ints: 2
  ints: 2
  type: INTS
}

input: "import/pool2/MaxPool:0"
input: "import/conv3/Variable:0"
output: "import/conv3/Conv2D:0"
name: "import/conv3/Conv2D"
op_type: "

Most DNN models spend a significant time in the Conv operators, likely to be around 60%~90%. To estimate the inference time, we need to figure out the toal number of multiply operations of all Conv operators required in a model. Do you know how to calculate the number of required number of multiply operations of all Conv operators in an ONNX model? We will give you a hint here and ask you to implement a script to generate the statistics in the homework section. The next script demonstrate how to figure out the input tensor sizes and dimensions of a Conv operator.

In [3]:
## parse_model.py

import onnx

onnx_model = onnx.load('./lenet.onnx')

## need to run shape inference in order to get a full value_info list
onnx_model = onnx.shape_inference.infer_shapes(onnx_model)

## List all tensor names in the raph
input_nlist = [k.name for k in onnx_model.graph.input]
initializer_nlist = [k.name for k in onnx_model.graph.initializer]
value_info_nlist = [k.name for k in onnx_model.graph.value_info]

print('\ninput list: {}'.format(input_nlist))
print('\ninitializer list: {}'.format(initializer_nlist))
print('\nvalue_info list: {}'.format(value_info_nlist))

## a simple function to calculate the tensor size and extract dimension information
def get_size(shape):
    dims = []
    ndim = len(shape.dim)
    size = 1;
    for i in range(ndim):
        size = size * shape.dim[i].dim_value
        dims.append(shape.dim[i].dim_value)
    return dims, size

## find all `Conv` operators and print its input information
for i in onnx_model.graph.node:
    if (i.op_type == 'Conv'):
        print('\n-- Conv "{}" --'.format(i.name))
        for j in i.input:
            if j in input_nlist:
                idx = input_nlist.index(j)
                (dims, size) = get_size(onnx_model.graph.input[idx].type.tensor_type.shape)
                print('input {} has {} elements dims = {}'.format(j, size, dims  ))
            elif j in initializer_nlist:
                idx = initializer_nlist.index(j)
                (dims, size) = get_size(onnx_model.graph.initializer[idx].type.tensor_type.shape)
                print('input {} has {} elements dims = {}'.format(j, size, dims))
            elif j in value_info_nlist:
                idx = value_info_nlist.index(j)
                (dims, size) = get_size(onnx_model.graph.value_info[idx].type.tensor_type.shape)
                print('input {} has {} elements dims = {}'.format(j, size, dims))



input list: ['import/Placeholder:0', 'import/conv3/BiasAdd__19', 'import/conv3/Variable_1:0', 'import/conv2/Variable_1:0', 'import/conv4last/Variable_1:0', 'import/conv1first/Variable_1:0', 'import/conv3/Variable:0', 'import/conv4last/BiasAdd__21', 'import/conv2/Variable:0', 'import/conv4last/Variable:0', 'import/conv1first/BiasAdd__15', 'import/conv1first/Variable:0', 'import/conv2/BiasAdd__17']

initializer list: ['import/conv3/BiasAdd__19', 'import/conv3/Variable_1:0', 'import/conv2/Variable_1:0', 'import/conv4last/Variable_1:0', 'import/conv1first/Variable_1:0', 'import/conv3/Variable:0', 'import/conv4last/BiasAdd__21', 'import/conv2/Variable:0', 'import/conv4last/Variable:0', 'import/conv1first/BiasAdd__15', 'import/conv1first/Variable:0', 'import/conv2/BiasAdd__17']

value_info list: ['import/conv4last/BiasAdd__22:0', 'import/conv1first/BiasAdd__16:0', 'import/conv3/BiasAdd__20:0', 'import/conv2/BiasAdd__18:0', 'import/conv1first/Conv2D:0', 'import/conv1first/BiasAdd:0', 'import

### Lab 2-3-2 Extracting Hidden State Tensors Using Hooks in PyTorch

In [4]:
import torchvision.models as models
import torch
activation = {}
# Define a hook function
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

# Load a pre-trained AlexNet model
model = models.alexnet(pretrained=True)
model.eval()

/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/envs/tensorflow/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /home/Hank/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth
100%|██████████| 233M/233M [00:17<00:00, 13.6MB/s] 


AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [5]:
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook


# Dictionary to store activations from each layer
activation = {}

# Register hook to each linear layer
for layer_name, layer in model.named_modules():
    if isinstance(layer, torch.nn.Linear):
        # Register forward hook
        layer.register_forward_hook(get_activation(layer_name))

# Run model inference
data = torch.randn(1, 3, 224, 224)
output = model(data)

# Access the saved activations
for layer in activation:
    print(f"Activation from layer {layer}: {activation[layer].shape}")

Activation from layer classifier.1: torch.Size([1, 4096])
Activation from layer classifier.4: torch.Size([1, 4096])
Activation from layer classifier.6: torch.Size([1, 1000])


### Lab 2-3-3 Model Computation Requirement Analysis

In [6]:
import torch.nn.functional as F
import torch.nn as nn

class SimpleLinearModel(nn.Module):
    def __init__(self):
        super(SimpleLinearModel,self).__init__()
        self.fc1 = nn.Linear(in_features=10, out_features=20, bias=False)
        self.fc2 = nn.Linear(in_features=20, out_features=15, bias=False)
        self.fc3 = nn.Linear(in_features=15, out_features=1, bias=False)
    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        F.relu(x)
        x = self.fc3(x)
        return x

linear_model = SimpleLinearModel()
sample_data = torch.randn(1, 10)

In [7]:
class SimpleConv(nn.Module):
    def __init__(self):
        super(SimpleConv, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.fc =  nn.Linear(in_features=32*28*28, out_features=10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = x.view(x.shape[0], -1)
        x = self.fc(x)
        return x

x = torch.rand(1, 1, 28, 28)
conv_model = SimpleConv()

#### Tutorial: Calculating Operations for AlexNet


In [8]:
import torch
import torchvision.models as models
import torch.nn as nn

model = models.alexnet(pretrained=True)

In [9]:
def calculate_output_shape(input_shape, layer):
    # Calculate the output shape for Conv2d, MaxPool2d, and Linear layers
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d)):
        kernel_size = (
            layer.kernel_size
            if isinstance(layer.kernel_size, tuple)
            else (layer.kernel_size, layer.kernel_size)
        )
        stride = (
            layer.stride
            if isinstance(layer.stride, tuple)
            else (layer.stride, layer.stride)
        )
        padding = (
            layer.padding
            if isinstance(layer.padding, tuple)
            else (layer.padding, layer.padding)
        )
        dilation = (
            layer.dilation
            if isinstance(layer.dilation, tuple)
            else (layer.dilation, layer.dilation)
        )

        output_height = (
            input_shape[1] + 2 * padding[0] - dilation[0] * (kernel_size[0] - 1) - 1
        ) // stride[0] + 1
        output_width = (
            input_shape[2] + 2 * padding[1] - dilation[1] * (kernel_size[1] - 1) - 1
        ) // stride[1] + 1
        return (
            layer.out_channels if hasattr(layer, "out_channels") else input_shape[0],
            output_height,
            output_width,
        )
    elif isinstance(layer, nn.Linear):
        # For Linear layers, the output shape is simply the layer's output features
        return (layer.out_features,)
    else:
        return input_shape


def calculate_macs(layer, input_shape, output_shape):
    # Calculate MACs for Conv2d and Linear layers
    if isinstance(layer, nn.Conv2d):
        kernel_ops = (
            layer.kernel_size[0]
            * layer.kernel_size[1]
            * (layer.in_channels / layer.groups)
        )
        output_elements = output_shape[1] * output_shape[2]
        macs = int(kernel_ops * output_elements * layer.out_channels)
        return macs
    elif isinstance(layer, nn.Linear):
        # For Linear layers, MACs are the product of input features and output features
        macs = int(layer.in_features * layer.out_features)
        return macs
    else:
        return 0

In [11]:
# Initial input shape
input_shape = (3, 224, 224)
total_macs = 0

# Iterate through the layers of the model
for name, layer in model.named_modules():
    if isinstance(layer, (nn.Conv2d, nn.MaxPool2d, nn.ReLU, nn.Linear)):
        output_shape = calculate_output_shape(input_shape, layer)
        macs = calculate_macs(layer, input_shape, output_shape)
        total_macs += macs
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: {macs}"
            )
        elif isinstance(layer, nn.MaxPool2d):
            # Also print shape transformation for MaxPool2d layers (no MACs calculated)
            print(
                f"Layer: {name}, Type: {type(layer).__name__}, Input Shape: {input_shape}, Output Shape: {output_shape}, MACs: N/A"
            )
        input_shape = output_shape  # Update the input shape for the next layer

print(f"Total MACs: {total_macs}")

Layer: features.0, Type: Conv2d, Input Shape: (3, 224, 224), Output Shape: (64, 55, 55), MACs: 70276800
Layer: features.2, Type: MaxPool2d, Input Shape: (64, 55, 55), Output Shape: (64, 27, 27), MACs: N/A
Layer: features.3, Type: Conv2d, Input Shape: (64, 27, 27), Output Shape: (192, 27, 27), MACs: 223948800
Layer: features.5, Type: MaxPool2d, Input Shape: (192, 27, 27), Output Shape: (192, 13, 13), MACs: N/A
Layer: features.6, Type: Conv2d, Input Shape: (192, 13, 13), Output Shape: (384, 13, 13), MACs: 112140288
Layer: features.8, Type: Conv2d, Input Shape: (384, 13, 13), Output Shape: (256, 13, 13), MACs: 149520384
Layer: features.10, Type: Conv2d, Input Shape: (256, 13, 13), Output Shape: (256, 13, 13), MACs: 99680256
Layer: features.12, Type: MaxPool2d, Input Shape: (256, 13, 13), Output Shape: (256, 6, 6), MACs: N/A
Layer: classifier.1, Type: Linear, Input Shape: (256, 6, 6), Output Shape: (4096,), MACs: 37748736
Layer: classifier.4, Type: Linear, Input Shape: (4096,), Output Shap

### Lab 2-3-5 Profiling with PyTorch

In [12]:
import torch
import torchvision.models as models
from torch.profiler import profile, record_function, ProfilerActivity

In [13]:
model = models.alexnet(pretrained=True)
model.eval()  # Set the model to evaluation mode
inputs = torch.randn(5, 3, 224, 224)

In [14]:
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

STAGE:2024-02-25 16:11:06 248:248 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2024-02-25 16:11:06 248:248 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2024-02-25 16:11:06 248:248 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


In [15]:
print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))


---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                             Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg    # of Calls  
---------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  
                  model_inference        11.83%       5.656ms       100.00%      47.811ms      47.811ms             1  
                     aten::linear         0.05%      24.000us        42.79%      20.457ms       6.819ms             3  
                      aten::addmm        42.56%      20.348ms        42.65%      20.392ms       6.797ms             3  
                     aten::conv2d         0.76%     362.000us        38.95%      18.624ms       3.725ms             5  
                aten::convolution         0.29%     137.000us        38.20%      18.262ms       3.652ms             5  
               aten::_convolution       

In [ ]:
with profile(activities=[ProfilerActivity.CPU], profile_memory=True, record_shapes=True) as prof:
    model(inputs)

print(prof.key_averages().table(sort_by="self_cpu_memory_usage", row_limit=10))

### Lab 2-3-6 Analyzing Profiling Results Using TensorBoard

In [ ]:
import torch.optim
import torch.profiler
import torch.utils.data
import torchvision.datasets
import torchvision.models
import torchvision.transforms as T


transform = T.Compose(
    [T.Resize(224),
     T.ToTensor(),
     T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)

device = torch.device("cpu")
model = torchvision.models.resnet18(weights='IMAGENET1K_V1')
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
model.train()

In [ ]:
def train(data):
    inputs, labels = data[0].to(device=device), data[1].to(device=device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

In [ ]:
with torch.profiler.profile(
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=1),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./log/resnet18'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof:
    for step, batch_data in enumerate(train_loader):
        prof.step()  # Need to call this at each step to notify profiler of steps' boundary.
        if step >= 1 + 1 + 3:
            break
        train(batch_data)